# Singular Value Projection
Results: (with rank k = 3, 10 iterations)
- Score: 0.98734 on Kaggle
- 0.864232130320197 on training set

## Packages

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from utils import import_data_to_matrix, extract_submission
from utils import NUMBER_OF_MOVIES, NUMBER_OF_USERS
from utils import zscore_masked_items

## Data Preprocessings
- Extract data to row-column format
- Impute missing data with 0
- Normalize item by item

- Rating matrix A

In [14]:
A = import_data_to_matrix()

- Observation matrix Ω
- Normalize item by item (z-scores)
$$A_{ij} = \frac{A_{ij} - \overline{A{j}}}{std(A_{j})}$$

Important: 
- Only observed entries are updated
- Mean and std is computed only over observed entries

In [15]:
W = (A > 0).astype(int)
norm_A, mean_A, stddev_A = zscore_masked_items(A, W)

## Projected gradient descent
$$A^0 = 0$$
$$A^{t+1} = [A^t + \eta\Pi_{\Omega}(A - A^t)]_k$$

In [19]:
def rmse(A, A_t, W):
    return np.sum(W*((A-A_t)**2)) / np.sum(W)

k = 3
iter = 1
A_t = np.zeros((NUMBER_OF_USERS , NUMBER_OF_MOVIES))
eta = 5.0
for epoch in tqdm(range(10)):
    residual = norm_A - A_t
    temp = A_t + eta * W * residual

    # SVD
    U, s, Vt = np.linalg.svd(temp, full_matrices=False)
    s[k+1:] = 0
    # Reconstruction - project temp to space of rank k matrices using SVD computed above
    A_t = np.dot(U * s, Vt)

    iter+=1
    eta = eta / (iter)**(1/2)
    print("k = ", k, ", RMSE = ", rmse(norm_A, A_t, W))

 10%|█         | 1/10 [00:01<00:15,  1.69s/it]

k =  3 , RMSE =  0.9770562152608658


 20%|██        | 2/10 [00:03<00:13,  1.69s/it]

k =  3 , RMSE =  0.9454868226581579


 30%|███       | 3/10 [00:05<00:12,  1.80s/it]

...

k =  3 , RMSE =  0.8642559854386712


 90%|█████████ | 9/10 [00:16<00:02,  2.02s/it]

k =  3 , RMSE =  0.8642378486024922


100%|██████████| 10/10 [00:19<00:00,  1.91s/it]

k =  3 , RMSE =  0.864232130320197


- Undo the normalization.

In [21]:
rec_A = A_t
#undo normalization
for j in range(1000):
    rec_A[:,j] *= stddev_A[j]
    rec_A[:,j] += mean_A[j]

In [22]:
print(rec_A)

[[3.34314789 3.48514607 3.43280897 ... 3.24781903 3.3195405  3.64296511]
 [3.26259308 3.49728658 3.24904276 ... 3.346204   3.26395315 3.54137883]
 [3.20626646 3.420788   3.29987276 ... 3.15375544 3.09871067 3.23903676]
 ...
 [3.33355334 3.44794827 3.50192241 ... 3.13525845 3.24685323 3.49788625]
 [3.34005564 3.48865605 3.45480456 ... 3.23155588 3.28548591 3.53257135]
 [3.62937799 3.75831291 3.74855674 ... 3.53885774 3.62052528 3.72692057]]


## Export Predictions


In [23]:
extract_submission(rec_A, file="baseline")